In [32]:
import numpy as np
import math
import csv
from proj1_helpers import *
from Hadrien import *
from ML_methods import *
from features_engineering import *
import matplotlib.pyplot as plt


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
y, tx, ids = load_csv_data("data_csv/train.csv", sub_sample=True)

In [34]:
print("y has shape {0}".format(y.shape))
print("X has shape {0}".format(tx.shape))

y has shape (5000,)
X has shape (5000, 30)


#### NAN processing
-> mets les NAN a la moyenne

In [35]:
tx_nan = transform_to_nan(tx, -900)
tx_nan_mean, mean =transform_to_mean(tx_nan)

# function prediction
-> function that tests the different methods and returns accuracy

In [38]:
def prediction(x, y, split, max_iters, gamma, type_func):
    #split data
    x_tr,x_te,y_tr,y_te=split_data(x,y,split)

    if(type_func == 'gd'):
        initial_w_gd = np.zeros(x.shape[1])
        losses, ws = gradient_descent(y_tr, x_tr, initial_w_gd, max_iters, gamma)
    elif(type_func == 'sgd'):
        initial_w_sgd = np.zeros(x.shape[1])
        losses, ws = stochastic_gradient_descent(y_tr, x_tr, initial_w_sgd, 10, max_iters, gamma)
    elif(type_func == 'ls'):
        ws = least_squares(y, x)
        losses = compute_loss(y, x, ws)
    elif(type_func == 'ridge'):
        #lambdas = np.logspace(-100, 100, 200)
        #losses = []
        #for index, lambda_ in enumerate(lambdas):
            #ws = ridge_regression(y, x, lambda_)
            #losses.append(compute_loss(y, x, ws))
        lambdas = np.logspace(-100, 100, 200)
        losses = []
        ws = []
        for index, lambda_ in enumerate(lambdas):
            ws.append(ridge_regression(y, x, lambda_))
            losses.append(compute_loss(y, x, ws[index]))
       
    #losses
    l,wnew=minimum_loss_vector(losses, ws)

    #predictions
    y_pred=predict_labels(wnew, x_tr)

    #accuracy
    print(accuracy_calculator(y_pred, y_tr))

## Basic Data
-> test the data set without any feature processing

In [39]:
x_tr,x_te,y_tr,y_te = split_data(tx_nan_mean,y,0.8)
prediction(x_tr, y_tr, 0.8, 1000, 0.000001, 'gd')
prediction(x_tr, y_tr, 0.8, 1000, 0.000001, 'sgd')
#prediction(x_tr, y_tr, 0.8, 1000, 0.000001, 'ls')
prediction(x_tr, y_tr, 0.8, 1000, 0.000001, 'ridge')

0.7125
0.7046875
0.75375


### Engineered feature 22

In [55]:
#test of feature engineering depending on classification by feature 22
tx_22plus0 = new_feature_PRI_jet_num(tx_nan_mean, 0.0)
tx_22plus1 = new_feature_PRI_jet_num(tx_22plus0, 1.0)
tx_22plus2 = new_feature_PRI_jet_num(tx_22plus1, 2.0)
tx_22plus3 = new_feature_PRI_jet_num(tx_22plus2, 3.0)

print(np.shape(tx_22plus3))

prediction(tx_22plus3, y, 0.8, 1000, 0.0000015, 'gd')
prediction(tx_22plus3, y, 0.8, 1000, 0.0000015, 'sgd')
#prediction(tx_22plus, y, 0.8, 1000, 0.067, 'ls')
prediction(tx_22plus3, y, 0.8, 1000, 0.0000015, 'ridge')

(5000, 34)
0.71825
0.71525
0.75725


### Log transformation of features that have an exponential distribution

In [65]:
features_log = [0, 2, 3, 5, 9, 13, 16, 23, 29]
tx_log = transform_feature_cos(tx_22plus3, features_log)
prediction(tx_log, y, 0.8, 1000, 0.00001, 'gd')
prediction(tx_log, y, 0.8, 1000, 0.0000015, 'sgd')
#prediction(tx_log, y, 0.8, 1000, 0.0000015, 'ls')
prediction(tx_log, y, 0.8, 1000, 0.0000015, 'ridge')

0.7015
0.7
0.73825


### Angles in cos
-> features that correspond to angles are transformed into cos after log transformation

In [58]:
features_cos = [15, 18, 20, 25, 28]
tx_cos3 = transform_feature_cos(tx_22plus3, features_cos)
prediction(tx_cos3, y, 0.8, 1000, 0.0000015, 'gd')
prediction(tx_cos3, y, 0.8, 1000, 0.0000015, 'sgd')
#prediction(tx_cos3, y, 0.8, 1000, 0.0000015, 'ls')
prediction(tx_cos3, y, 0.8, 1000, 0.0000015, 'ridge')

0.71825
0.71525
0.75675
